In [4]:
import os
import openai
from dotenv import load_dotenv
load_dotenv()

openai.organization = "org-******"
openai.api_key = os.getenv('OPENAI_API_KEY')

# 3.0 LLM and Chat Models Base 만들기

In [6]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

llm = OpenAI(model="gpt-3.5-turbo-instruct") 
chat = ChatOpenAI(model_name="gpt-3.5-turbo")

# llm = OpenAI( open_api_key="sk-zzz")
# chat = ChatOpenAI(open_api_key="sk-zzz")

a = llm.predict("How many planets are there?") # text-davinch
b = chat.predict("How many planets are there?") # gpt-3

a, b

('\n\nAs of now, there are 8 planets in our solar system: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune. However, there may be more planets in our solar system that have not yet been discovered.',
 'There are eight recognized planets in our solar system: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune.')

# 3.1 Predcit Message Model 만들기

In [7]:
chat = ChatOpenAI(
    temperature=0.1 # 모델에게 창의성 부여, 숫자가 낮을수록 모델의 창의성이 낮음
)

In [13]:
# message를 predict
from langchain.schema import HumanMessage, AIMessage, SystemMessage
# HumanMessage = 인간메시지, AIMessage = AI에 의해 보내지는 것, SystemMessage  = LLM에 설정들을 제공하기 위한 메시지

messages = [
    SystemMessage(content="You are a geography expert. And you only reply in Italian.",
                  # You are a geography expert. And you only reply in {language}.
                  ), # AI에 일종의 기본 설정, 기본 값, 기본 context 설정 (가상의 시나리오)
    AIMessage(content="Ciao, mi chiamo Paolo!"), # string을 미리 만들어 두고, 일종의 가상 대화를 만듦
    # AI에게 자격을 부여해 주는 메시지
    # Ciao, mi chiamo {name}!
    HumanMessage(content="What is the distence between Mexico an Thailand. Also, What is your name?",),
    # 내가 사용자로서 질문을 함
    # What is the distence between {country_a} an {country_b}. Also, What is your name?
]

chat.predict_messages(messages)

AIMessage(content='Ciao! La distanza tra il Messico e la Thailandia è di circa 16.000 chilometri. Il mio nome è Paolo. Posso aiutarti con altre domande di geografia?')

# 3.2 Prompt Templates

In [21]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate

template = PromptTemplate.from_template("What is the distence between {country_a} an {country_b}",
                                        )

chat = ChatOpenAI(temperature=0.1)

template.format(country_a="Mexico", country_b="Thailand")

'What is the distence between Mexico an Thailand'

In [23]:
chat = ChatOpenAI(temperature=0.1)

template = PromptTemplate.from_template("What is the distence between {country_a} an {country_b}",
                                        )
prompt = template.format(country_a="Mexico", country_b="Thailand")

chat.predict(prompt)

'The distance between Mexico and Thailand is approximately 9,500 miles (15,300 kilometers) when measured in a straight line.'

In [29]:
template = ChatPromptTemplate.from_messages([
    ("system", "You are a geography expert. And you only reply in {language}."),
    ("ai", "Ciao, mi chiamo {name}!"),
    ("human", "What is the distence between {country_a} an {country_b}. Also, What is your name?",),
])

prompt = template.format_messages(
    language="Greek",
    name="Socrates",
    country_a="Mexico",
    country_b="Thailand"
)

chat.predict_messages(prompt)

AIMessage(content='Γεια σας! Η απόσταση μεταξύ του Μεξικού και της Ταϊλάνδης είναι περίπου 16.000 χιλιόμετρα. Το όνομά μου είναι Σωκράτης. Πώς μπορώ να βοηθήσω;')

# 3.3 Output Parser and LCEL

## Prompt > Output Parser

In [30]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

chat = ChatOpenAI(temperature=0.1)

# Output Parser가 필요한 이유는 LLM의 응답을 변형해야 할 때가 있기 때문
# LLM은 텍스트 응답이기 때문에 list 형태

In [35]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    # OutputParser는 parse라는 함수를 지정해야함
    def parse(self, text): # text 매개변수를 통해 전체 텍스트를 입력 받음
        items =  text.strip().split(",") # .strip() = 공백 제거
        return list(map(str.strip,items)) # list를 반환하기 전 먼저 items의 각 요소에 작업해 줌

p = CommaOutputParser()

p.parse("Hello, how, are, you")
# ['Hello', 'how', 'are', 'you']

['Hello', 'how', 'are', 'you']

In [43]:
template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items}. Do NOT reply with anything else.",),
        ("human", "{question}"),
    ]
)

prompt = template.format_messages(
    max_items=10, # 10개 이하
    question="What are the planets?"
)

chat.predict_messages(prompt)

AIMessage(content='Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, Neptune, Pluto')

In [47]:
template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items}. Do NOT reply with anything else.",),
        ("human", "{question}"),
    ]
)

prompt = template.format_messages(
    max_items=10, # 10개 이하
    question="What are the colors?"
)

result = chat.predict_messages(prompt)

p = CommaOutputParser()

p.parse(result.content)

['Red',
 'blue',
 'green',
 'yellow',
 'orange',
 'purple',
 'pink',
 'black',
 'white',
 'brown']

In [46]:
template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items} in lowercase. Do NOT reply with anything else.",),
        ("human", "{question}"),
    ]
)

prompt = template.format_messages(
    max_items=10, # 10개 이하
    question="What are the planets?"
)

result = chat.predict_messages(prompt)

p = CommaOutputParser()

p.parse(result.content)

['mercury', 'venus', 'earth', 'mars', 'jupiter', 'saturn', 'uranus', 'neptune']

### Chain 만들기

In [50]:
# chain = 모든 요소를 합쳐주는 역할
# 합쳐진 요소들은 하나의 chain으로 실행. 하나하나 순서대로, result를 반환할 때까지

chat = ChatOpenAI(temperature=0.1)
# chat의 input type은 string 한 개, 메시지의 list 또는 promptValue
# ChatModel은 ChatMessage를 반환하고, 그 값은 Ouput Parser에게 전해짐

class CommaOutputParser(BaseOutputParser):
    # OutputParser는 parse라는 함수를 지정해야함
    def parse(self, text): # text 매개변수를 통해 전체 텍스트를 입력 받음
        items =  text.strip().split(",") # .strip() = 공백 제거
        return list(map(str.strip,items)) # list를 반환하기 전 먼저 items의 각 요소에 작업해 줌
# Output Parser는 LLM 이나 ChatModel의 반환타입과 동일함
# = Output Parser의 Input type = ChatMessage's input type

template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items}. Do NOT reply with anything else.",),
        ("human", "{question}"),
    ]
)

chain = template | chat | CommaOutputParser()
# template에 chian.invoke가 전달되고, chat에 template 가 전달되고 Chat의 반환값은 OutputParser에게 전달됨
# Chain은 단지 component의 그룹 혹은 나열
# component들은 입력값을 받아서 출력값을 반환함

chain.invoke({
    "max_items":5,
    "question":"포켓몬에는 뭐가 있어?"
})
# Prompt의 type은 dictionary

['피카츄', '꼬부기', '파이리', '이상해씨']

# 3.4 Chaining Chains

In [10]:
from langchain.chat_models import ChatAnthropic, ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(temperature=0.1, 
                  streaming=True, 
                  callbacks=[StreamingStdOutCallbackHandler()])
# streaming은 우리가 model의 응답이 생성되는걸 볼 수 있게 해줌
# callbacks 덕분에 콘솔에서 응답의 진행을 볼 수 있게 됨

chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world-class international chef. You create easy to follow recipies for any type of cuisine with easy to fine ingredients."),
    ("human", "I want to cook {cuisine} food."),
])

chef_chain = chef_prompt | chat



In [11]:
veg_chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a vegetarian chef specialized on making traditional recipies vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to replace it"),
    ("human", "{recipe}")
])

veg_chain = veg_chef_prompt | chat

final_chain = {"recipe": chef_chain} | veg_chain
# chain에 context object를 넣어줌! (dictionary 형태)
# veg_chain보다 먼저 실행됨
# {"recipe": chef_chain} = LangChain 언어 = RunnableMap

final_chain.invoke({
    "cuisine": "indian"
})

# final_chain의 invoke 값은 chef_chain의 입력값인 cuisine으로 전달됨
# chef_chain으로부터 얻은 결과는 reciope라는 key에 저장됨
# recipe key의 값은 veg_chain의 입력값으로 전달됨
    
# veg_chain.invoke({
#     "recipe": "chatmodel"
# })

Great choice! Let's make a delicious and easy Chicken Tikka Masala. Here's the recipe:

Ingredients:
- 1 lb boneless, skinless chicken breasts, cut into bite-sized pieces
- 1 cup plain yogurt
- 2 tablespoons lemon juice
- 2 tablespoons vegetable oil
- 1 onion, finely chopped
- 3 cloves garlic, minced
- 1 tablespoon ginger, minced
- 1 can (14 oz) tomato sauce
- 1 tablespoon garam masala
- 1 teaspoon ground cumin
- 1 teaspoon ground coriander
- 1 teaspoon paprika
- 1/2 teaspoon turmeric
- 1/2 teaspoon cayenne pepper (adjust to taste)
- Salt and pepper to taste
- Fresh cilantro, chopped (for garnish)
- Cooked rice or naan bread (for serving)

Instructions:
1. In a bowl, mix together the yogurt, lemon juice, 1 tablespoon of vegetable oil, half of the minced garlic, half of the minced ginger, and a pinch of salt. Add the chicken pieces and stir to coat. Cover and marinate in the refrigerator for at least 1 hour, or overnight for best results.

2. Preheat the oven to 400°F (200°C). Thread th

AIMessageChunk(content="To make a vegetarian version of Chicken Tikka Masala, we can replace the chicken with a suitable alternative. Here's how you can adapt the recipe:\n\nIngredients:\n- 1 lb of paneer or extra-firm tofu, cut into bite-sized pieces\n- 1 cup plain yogurt (you can use dairy-free yogurt if preferred)\n- 2 tablespoons lemon juice\n- 2 tablespoons vegetable oil\n- 1 onion, finely chopped\n- 3 cloves garlic, minced\n- 1 tablespoon ginger, minced\n- 1 can (14 oz) tomato sauce\n- 1 tablespoon garam masala\n- 1 teaspoon ground cumin\n- 1 teaspoon ground coriander\n- 1 teaspoon paprika\n- 1/2 teaspoon turmeric\n- 1/2 teaspoon cayenne pepper (adjust to taste)\n- Salt and pepper to taste\n- Fresh cilantro, chopped (for garnish)\n- Cooked rice or naan bread (for serving)\n\nInstructions:\n1. Instead of marinating chicken, marinate the paneer or tofu in the yogurt, lemon juice, 1 tablespoon of vegetable oil, half of the minced garlic, half of the minced ginger, and a pinch of sal